<a href="https://colab.research.google.com/github/Lucas-Buk/IMT/blob/main/Cancer_Explora%C3%A7%C3%A3o_KMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Notebook com o carregamento dos dados gerados pelo K-Means para sua exploração, visando o entendimento das relações dos clusters

---

#**Bibliotecas e instalações**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pickle

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# **Dicionário de variáveis**

*   **ESCOLARI**: Código para escolaridade do paciente (int = 1).

      1 – ANALFABETO

      2 – ENS. FUND. INCOMPLETO

      3 – ENS. FUND. COMPLETO

      4 – ENSINO MÉDIO

      5 – SUPERIOR

      9 – IGNORADA
*   **IDADE**: Idade do paciente (int = 3).
*   **SEXO**: Sexo do paciente (int = 1). 

      1 – MASCULINO

      2 – FEMININO
*   **UFNASC**: UF de nascimento (char = 2). Outras opções: SI - Sem informação; OP - Outro país.	
*   **UFRESID**: UF de residência (char = 2). Outras opções: OP - Outro país.
*   **IBGE**: Código da cidade de residência do paciente segundo IBGE com digito verificador (char = 7).
*   **CIDADE**: Cidade de residência do paciente (char = 200).
*   **CATEATEND**: Categoria de atendimento ao diagnóstico (int = 1). 

      1 - CONVENIO

      2 - SUS

      3 – PARTICULAR

      9 – SEM INFORMAÇÃO
*   **DTCONSULT**: Data da 1ª consulta (date = 10). Formato: DD/MM/YYYY	
*   **CLINICA**: Código da clinica (int = 2).

      1 – ALERGIA/IMUNOLOGIA

      2 – CIRURGIA CARDIACA

      3 – CIRURGIA CABEÇA E PESCOÇO

      4 – CIRURGIA GERAL

      5 – CIRURGIA PEDIATRICA

      6 – CIRURGIA PLASTICA

      7 – CIRURGIA TORAXICA

      8 – CIRURGIA VASCULAR

      9 – CLINICA MEDICA

      10 – DERMATOLOGIA

      11 – ENDOCRINOLOGIA

      12 – GASTROCIRURGIA
      
      13 – GASTROENTEROLOGIA
      
      14 – GERIATRIA
      
      15 – GINECOLOGIA
      
      16 – GINECOLOGIA / OBSTETRICIA
      
      17 – HEMATOLOGIA
      
      18 – INFECTOLOGIA
      
      19 – NEFROLOGIA
      
      20 – NEUROCIRURGIA
      
      21 – NEUROLOGIA
      
      22 – OFTALMOLOGIA
      
      23 – ONCOLOGIA CIRURGICA
      
      24 – ONCOLOGIA CLINICA
      
      25 – ONCOLOGIA PEDIATRICA
      
      26 – ORTOPEDIA
      
      27 – OTORRINOLARINGOLOGIA
      
      28 – PEDIATRIA
      
      29 – PNEUMOLOGIA
      
      30 – PROCTOLOGIA
      
      31 – RADIOTERAPIA
      
      32 – UROLOGIA
      
      33 – MASTOLOGIA
      
      34 – ONCOLOGIA CUTANEA
      
      35 – CIRURGIA PELVICA
      
      36 – CIRURGIA ABDOMINAL
      
      37 – ODONTOLOGIA
      
      38 – TRANSPLANTE HEPATICO
      
      99 – IGNORADO	
*   **DIAGPREV**: Diagnóstico e tratamento anterior (int = 1).

      1 – SEM DIAGNÓSTICO / SEM TRATAMENTO
      
      2 – COM DIAGNÓSTICO / SEM TRATAMENTO
      
      3 – COM DIAGNÓSTICO / COM TRATAMENTO
      
      4 – OUTROS	
*   **DTDIAG**: Data do diagnóstico (date = 10). Formato: DD/MM/YYYY	
*   **BASEDIAG**: Código da base do diagnóstico (int = 1).
      
      1 – EXAME CLINICO
      
      2 – RECURSOS AUXILIARES NÃO MICROSCÓPICOS
      
      3 – CONFIRMAÇÃO MICROSCÓPICA
      
      4 – SEM INFORMAÇÃO	
*   **TOPO**: Código da topografia (char = 4). Formato: C999 	
*   **TOPOGRUP**: Grupo da topografia	(char = 3). Formato: C99	
*   **DESCTOPO**: Descrição da Topografia (char = 80).
*   **MORFO**: Código da morfologia (char = 5). Formato: 99999
*   **DESCMORFO**: Descrição da morfologia (char = 80).
*   **EC**: Estádio clínico (char = 5).
*   **ECGRUP**: Grupo do estadiamento clínico (char = 3).

      0 - Tumores primários, classificados como in situ

      I - Tumores localizados

      II - Tumores com envolvimento regional por extensão direta

      III - Tumores com envolvimento regional de linfonodos
      
      IV - Tumores com metástase à distância 

      X - Para tumores não avaliados pelo profissional responsável ou sem informação sobre estadiamento anotada no prontuário

      Y - Para tumores em que não se aplica a classificação TNM. São os tumores não sólidos (por exemplo, as leucemias)
*   **T**: Classificação TNM - T (char = 5).
*   **N**: Classificação TNM - N (char = 5).	
*   **M**: Classificação TNM - M (char = 3).	
*   **PT**: Estadiamento pós cirúrgico (char = 5).
*   **PN**: Estadiamento pós cirúrgico (char = 5).	
*   **PM**: Estadiamento pós cirúrgico (char = 3).	
*   **S**: Classificação TNM - S (int = 1). Domínio: 0; 1; 2; 3; 8 – NÃO SE APLICA; 9 – X
*   **G**: Classificação TNM – G (Grau) (char = 5). 

      Domínio (exceto C40, C41, C381, C382, C383, C47, C48 e C49):
      0; 1; 2; 3; 4; 8 – NÃO SE APLICA; 9 – X

      Domínio (somente C40, C41, C381, C382, C383, C47, C48 e C49):
      ALTO; BAIXO; 8 – NÃO SE APLICA; 9 – X
	
*   **LOCALTNM**: Classificação TNM - Localização (int = 1).
      
      1 – SUPERIOR
      
      2 – MEDIO
      
      3 – INFERIOR
      
      8 – NÃO SE APLICA
      
      9 – X	
*   **IDMITOTIC**: Classificação TNM – Índice Mitótico (int = 1).
      
      1 – ALTA
      
      2 – BAIXA
      
      8 – NÃO SE APLICA
      
      9 – X	
*   **PSA**: Classificação TNM - PSA (int = 1).
      
      1 – MENOR QUE 10
      
      2 – MAIOR OU IGUAL A 10 E MENOR QUE 20
      
      3 – MAIOR OU IGUAL A 20
      
      8 – NÃO SE APLICA
      
      9 – X	
*   **GLEASON**: Classificação TNM - Gleason (int = 1).
      
      1 – MENOR OU IGUAL A 6
      
      2 – IGUAL A 7
      
      3 – MAIOR OU IGUAL A 8
      
      8 – NÃO SE APLICA
      
      9 – X	
*   **OUTRACLA**: Outra classificação de estadiamento (char = 20).
*   **META01**: Metástase (char = 3).	Formato: C99
*   **META02**: Metástase (char = 3).	Formato: C99
*   **META03**: Metástase (char = 3).	Formato: C99
*   **META04**: Metástase (char = 3).	Formato: C99
*   **DTTRAT**: Data de inicio do tratamento (date = 10). Formato: DD/MM/YYYY	
*   **NAOTRAT**: Código da razão para não realização do tratamento (int = 1).
      
      1 – RECUSA DO TRATAMENTO
      
      2 – DOENÇA AVANÇADA, FALTA DE CONDIÇÕES CLINICAS
      
      3 – OUTRAS DOENÇAS ASSOCIADAS
      
      4 – ABANDONO DE TRATAMENTO
      
      5 – OBITO POR CANCER
      
      6 – OBITO POR OUTRAS CAUSAS, SOE
      
      7 – OUTRAS
      
      8 – NÃO SE APLICA (CASO TENHA TRATAMENTO)
      
      9 – SEM INFORMAÇÃO	
*   **TRATAMENTO**: Código de combinação dos tratamentos realizados (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento
      
      J – Nenhum tratamento realizado	
*   **TRATHOSP**: Código de combinação dos tratamentos realizados no hospital (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento

      J – Nenhum tratamento realizado	
*   **TRATFANTES**: Código de combinação dos tratamentos realizados antes/durante admissão fora do hospital (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento
      
      J – Nenhum tratamento realizado
      
      K – Sem informação
*   **TRATFAPOS**: Código de combinação dos tratamentos realizados após admissão fora do hospital (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento
      
      J – Nenhum tratamento realizado
      
      K – Sem informação
*   **NENHUM**: Tratamento recebido no hospital = nenhum (int = 1). 0 – NÃO; 1 – SIM
*   **CIRURGIA**: Tratamento recebido no hospital = cirurgia (int = 1). 0 – NÃO; 1 – SIM
*   **RADIO**: Tratamento recebido no hospital = radioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **QUIMIO**: Tratamento recebido no hospital = quimioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **HORMONIO**: Tratamento recebido no hospital = hormonioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **TMO**: Tratamento recebido no hospital = tmo (int = 1). 0 – NÃO; 1 – SIM
*   **IMUNO**: Tratamento recebido no hospital = imunoterapia (int = 1). 0 – NÃO; 1 – SIM
*   **OUTROS**: Tratamento recebido no hospital = outros (int = 1). 0 – NÃO; 1 – SIM
*   **NENHUMANT**: Tratamento recebido fora do hospital e antes da admissão = nenhum (int = 1). 0 – NÃO; 1 – SIM
*   **CIRURANT**: Tratamento recebido fora do hospital e antes da admissão = cirurgia (int = 1). 0 – NÃO; 1 – SIM
*   **RADIOANT**: Tratamento recebido fora do hospital e antes da admissão = radioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **QUIMIOANT**: Tratamento recebido fora do hospital e antes da admissão = quimioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **HORMOANT**: Tratamento recebido fora do hospital e antes da admissão = hormonioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **TMOANT**: Tratamento recebido fora do hospital e antes da admissão = tmo (int = 1). 0 – NÃO; 1 – SIM
*   **IMUNOANT**: Tratamento recebido fora do hospital e antes da admissão = imunoterapia (int = 1). 0 – NÃO; 1 – SIM
*   **OUTROANT**: Tratamento recebido fora do hospital e antes da admissão = outros (int = 1). 0 – NÃO; 1 – SIM	
*   **NENHUMAPOS**: Tratamento recebido fora do hospital e durante/após admissão = nenhum	(int = 1). 0 – NÃO; 1 – SIM	
*   **CIRURAPOS**: Tratamento recebido fora do hospital e durante/após admissão = cirurgia	(int = 1). 0 – NÃO; 1 – SIM	
*   **RADIOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = radioterapia	(int = 1). 0 – NÃO; 1 – SIM	
*   **QUIMIOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = quimioterapia	(int = 1). 0 – NÃO; 1 – SIM		
*   **HORMOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = hormonioterapia	(int = 1). 0 – NÃO; 1 – SIM		
*   **TMOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = tmo	(int = 1). 0 – NÃO; 1 – SIM		
*   **IMUNOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = imunoterapia	(int = 1). 0 – NÃO; 1 – SIM	
*   **OUTROAPOS**: Tratamento recebido fora do hospital e durante/após admissão = outros	(int = 1). 0 – NÃO; 1 – SIM	
*   **DTULTINFO**: Data da última informação do paciente (date = 10). Formato: DD/MM/YYYY
*   **ULTINFO**: Última informação sobre o paciente (int = 1).
      
      1 – VIVO, COM CÂNCER
      
      2 – VIVO, SOE
      
      3 – OBITO POR CANCER
      
      4 – OBITO POR OUTRAS CAUSAS, SOE
*   **CONSDIAG**: Diferença em dias entre as datas de consulta o diagnóstico (num = dias).
*   **TRATCONS**: Diferença em dias entre as datas de consulta e tratamento (num = dias).	
*   **DIAGTRAT**: Diferença em dias entre as datas de tratamento e diagnóstico (num = dias).
*   **ANODIAG**: Ano de diagnóstico (int = 4). Formato: 9999
*   **CICI**: Tumor infantil (char = 5).	
*   **CICIGRUP**: Tumor infantil – Grupo (char = 80).	
*   **CICISUBGRU**: Tumor infantil – Sub grupo (char = 80).	
*   **FAIXAETAR**: Faixa etária do paciente (char = 5).	
*   **LATERALI**: Lateralidade (int = 1).
      
      1 – DIREITA
      
      2 – ESQUERDA
      
      3 – BILATERAL
      
      8 - NÃO SE APLICA	
*   **INSTORIG**: Instituição de origem (char = 200). Obrigatório somente se DIAGPREV = 03 – COM DIAGNÓSTICO / COM TRATAMENTO
*   **DRS**: Departamentos Regionais de Saúde (char = 200).
*   **RRAS**: RRAS (char = 200).	
*   **PERDASEG**: Perda de seguimento (int = 1). 
      
      0 – Não
      
      1 – Sim
      
      8 – Não se aplica (excluído do cálculo para o indicador perda de seguimento)	
*   **ERRO**: Admissão com erro (int = 1). 0 – Sem; 1 – Com
*   **DTRECIDIVA**: Data da última ocorrência de recidiva (date = 10). Formato: DD/MM/YYYY	
*   **RECNENHUM**: Sem recidiva (int = 1). 0 - Não; 1 - Sim
*   **RECLOCAL**: Recidiva local (int = 1). 0 - Não; 1 - Sim	
*   **RECREGIO**: Recidiva regional (int = 1). 0 - Não; 1 - Sim	
*   **RECDIST**: Recidiva a distância / metástase (int = 1). 0 - Não; 1 - Sim	
*   **REC01**: Local da recidiva/metástase (char = 3). Formato: C99 
*   **REC02**: Local da recidiva/metástase (char = 3). Formato: C99 	
*   **REC03**: Local da recidiva/metástase (char = 3). Formato: C99 
*   **REC04**: Local da recidiva/metástase (char = 3). Formato: C99 	
*   **IBGEATEN**: Código IBGE da instituição (int = 7).	
*   **CIDO**: Código da morfologia 3ª Edição (int = 5). Formato: 99999	
*   **DSCCIDO**: Descrição da morfologia 3ª Edição (char = 89).




# **Dados**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Trabalho/Cancer/Datasets/kmeans_preprocessing.csv')
df.head()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,GRUPO
0,4,40.0,2,3530805,9,15,1,3,222,45,81402,0,0,25,15,0,30,0,0,8,8,8,8,8,0,0,0,0,8,0,0,2,9,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,7,95.0,88.0,2000,23,5,4,8,14,15,1,1,0,0,0,0,0,0,0,3509502,17
1,9,45.0,2,3509502,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,8,8,8,8,8,8,0,0,0,0,8,5,5,2,9,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,12,15.0,3.0,2000,23,5,4,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
2,2,63.0,2,3509502,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,8,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,15.0,9.0,2000,23,5,6,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
3,9,64.0,2,3545803,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,8,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,33.0,27.0,2000,23,5,6,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
4,1,48.0,2,3530805,9,15,2,3,222,45,80703,19,3,25,15,0,41,22,8,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,132.0,132.0,2000,23,5,4,8,14,15,1,1,0,0,0,0,0,0,0,3509502,17


In [ ]:
with open('/content/drive/MyDrive/Trabalho/Cancer/Modelos/encoders_pp.pkl', 'rb') as handle:
  encoders = pickle.load(handle)

In [ ]:
df_aux = df.copy()
for col, le in encoders.items():
  df_aux[col] = le.inverse_transform(df_aux[col])

df_aux.head()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,GRUPO
0,4,40.0,2,3530805,9,15,1,3,C539,C53,81402,0,0,X,X,0,IS,0,0,8,8,8,8,8,0,0,0,0,8,A,A,J,J,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,7,95.0,88.0,2000,Sem informação,Sem informação,40-49,8,14,15,1,1,0,0,0,0,0,0,0,3509502,17
1,9,45.0,2,3509502,9,15,1,3,C539,C53,80703,IIIB,III,X,X,0,X,X,X,8,8,8,8,8,0,0,0,0,8,F,F,J,J,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,12,15.0,3.0,2000,Sem informação,Sem informação,40-49,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
2,2,63.0,2,3509502,9,15,1,3,C539,C53,80703,IIIB,III,X,X,0,X,X,X,8,8,8,8,8,0,0,0,0,8,B,B,J,J,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,15.0,9.0,2000,Sem informação,Sem informação,60-69,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
3,9,64.0,2,3545803,9,15,1,3,C539,C53,80703,IIIB,III,X,X,0,X,X,X,8,8,8,8,8,0,0,0,0,8,B,B,J,J,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,33.0,27.0,2000,Sem informação,Sem informação,60-69,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
4,1,48.0,2,3530805,9,15,2,3,C539,C53,80703,IIIB,III,X,X,0,X,X,X,8,8,8,8,8,0,0,0,0,8,B,B,J,J,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,132.0,132.0,2000,Sem informação,Sem informação,40-49,8,14,15,1,1,0,0,0,0,0,0,0,3509502,17


In [ ]:
k = df_aux.GRUPO.nunique()
dict_grup = {} # Dicionário com os grupos separados
for i in range(0, k):
  dict_grup[i] = df_aux[df_aux.GRUPO == i]

In [ ]:
n = 0 # Grupo para análise
df_grupo = dict_grup[n]
df_grupo.head()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,GRUPO
7,9,80.0,2,3509502,2,33,1,3,C504,C50,84803,IIA,II,2,0,0,2,0,0,8,8,8,8,8,0,0,0,0,8,D,D,J,J,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,8,98.0,90.0,2014,Sem informação,Sem informação,70+,1,7,15,1,1,0,0,0,0,0,0,0,3509502,0
39,2,58.0,2,3546702,9,33,1,3,C509,C50,85003,IIIA,III,3,1,0,2,1,0,8,8,8,8,8,0,0,0,0,8,E,E,J,J,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,40.0,34.0,2000,Sem informação,Sem informação,50-59,8,10,14,0,1,0,0,0,C34,0,0,0,3509502,0
43,4,66.0,2,3543907,9,15,2,3,C519,C51,80703,IVA,IV,X,X,0,2,0,0,8,8,8,8,8,0,0,0,0,8,A,A,J,J,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,4,0,25.0,25.0,2000,Sem informação,Sem informação,60-69,8,10,14,0,1,0,0,0,0,0,0,0,3509502,0
45,4,41.0,2,3509502,9,33,2,3,C509,C50,85003,I,I,1,0,0,1,0,0,8,8,8,8,8,0,0,0,0,8,D,D,J,J,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,41.0,41.0,2000,Sem informação,Sem informação,40-49,8,7,15,1,1,0,0,0,0,0,0,0,3509502,0
46,1,81.0,2,3552106,9,33,2,3,C509,C50,85003,I,I,1,0,0,1,0,0,8,8,8,8,8,0,0,0,0,8,A,A,J,J,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,104.0,104.0,2000,Sem informação,Sem informação,70+,8,7,16,1,1,0,0,0,0,0,0,0,3509502,0


# **Análise**

In [ ]:
df_grupo.describe()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,MORFO,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,NAOTRAT,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,IBGEATEN,GRUPO
count,138103.000000,138103.000000,138103.000000,1.381030e+05,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.0,138103.000000,138103.0,138103.0,138103.000000,138103.0,138103.0,138103.0,138103.000000,138103.000000,138103.000000,138103.0,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.000000,138103.0,138103.000000,138103.000000,1.381030e+05,138103.0
mean,4.633216,64.882834,1.470823,3.532731e+06,4.536491,17.395241,1.212841,2.998016,81907.798194,8.002571,8.000109,7.994504,8.003751,7.979378,7.998841,0.001521,0.971847,0.072069,0.105805,0.000101,0.000029,0.002136,0.0,0.999913,0.0,0.0,0.989015,0.0,0.0,0.0,0.000550,0.000014,0.000080,0.0,2.401114,38.698833,55.618850,38.602521,2010.652701,7.116956,6.505702,10.090751,0.122307,0.999088,0.0,0.000833,0.000014,3.537792e+06,0.0
std,3.052778,14.025152,0.499150,5.279688e+04,3.576209,12.215007,0.409318,0.177549,1975.125545,0.052046,0.010421,0.185240,0.073204,0.353456,0.047669,0.038965,0.165410,0.258604,0.307589,0.010068,0.005382,0.046169,0.0,0.009321,0.0,0.0,0.104230,0.0,0.0,0.0,0.023452,0.003806,0.008924,0.0,0.859735,77.077642,87.342786,72.654290,5.460195,2.234391,5.527511,9.959199,0.327642,0.030192,0.0,0.028845,0.003806,1.508213e+04,0.0
min,1.000000,0.000000,1.000000,2.111300e+06,1.000000,1.000000,1.000000,1.000000,80102.000000,4.000000,8.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,2000.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.502804e+06,0.0
25%,2.000000,56.000000,1.000000,3.522000e+06,2.000000,6.000000,1.000000,3.000000,80903.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,2.000000,1.000000,6.000000,0.000000,2006.000000,8.000000,1.000000,6.000000,0.000000,1.000000,0.0,0.000000,0.000000,3.525300e+06,0.0
50%,3.000000,66.000000,1.000000,3.540804e+06,2.000000,12.000000,1.000000,3.000000,80923.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,2.000000,19.000000,32.000000,0.000000,2011.000000,8.000000,6.000000,9.000000,0.000000,1.000000,0.0,0.000000,0.000000,3.549805e+06,0.0
75%,9.000000,75.000000,2.000000,3.550308e+06,9.000000,30.000000,1.000000,3.000000,82113.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,3.000000,47.000000,70.000000,57.000000,2015.000000,8.000000,10.000000,12.000000,0.000000,1.000000,0.0,0.000000,0.000000,3.550308e+06,0.0
max,9.000000,103.000000,2.000000,4.321204e+06,9.000000,99.000000,2.000000,9.000000,95813.000000,9.000000,9.000000,8.000000,9.000000,9.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,0.0,0.0,1.000000,0.0,0.0,0.0,1.000000,1.000000,1.000000,0.0,4.000000,1571.000000,1238.000000,1300.000000,2020.000000,8.000000,17.000000,99.000000,1.000000,1.000000,0.0,1.000000,1.000000,3.555000e+06,0.0


*   **DIAGPREV**: Diagnóstico e tratamento anterior (int = 1).

      1 – SEM DIAGNÓSTICO / SEM TRATAMENTO
      
      2 – COM DIAGNÓSTICO / SEM TRATAMENTO
      
      3 – COM DIAGNÓSTICO / COM TRATAMENTO
      
      4 – OUTROS

In [ ]:
df_grupo.DIAGPREV.value_counts()

1    108709
2     29394
Name: DIAGPREV, dtype: int64

In [ ]:
df_aux.DIAGPREV.value_counts()

1    593200
2    350459
Name: DIAGPREV, dtype: int64

*   **BASEDIAG**: Código da base do diagnóstico (int = 1).
      
      1 – EXAME CLINICO
      
      2 – RECURSOS AUXILIARES NÃO MICROSCÓPICOS
      
      3 – CONFIRMAÇÃO MICROSCÓPICA
      
      9 – SEM INFORMAÇÃO

In [ ]:
df_grupo.BASEDIAG.value_counts()

3    137433
2       370
1       213
9        87
Name: BASEDIAG, dtype: int64

In [ ]:
df_aux.BASEDIAG.value_counts()

3    927293
2     13320
1      2059
9       987
Name: BASEDIAG, dtype: int64

*   **TOPO**: Código da topografia (char = 4). Formato: C999 	

In [ ]:
df_grupo.TOPO.nunique()

221

In [ ]:
df_aux.TOPO.nunique()

331

* **TOPOGRUP**: Grupo da topografia (char = 3). Formato: C99

In [ ]:
df_grupo.TOPOGRUP.value_counts()#.head(10)

C44    77014
C18     8945
C50     7323
C61     5953
C73     4990
C16     4884
C20     3830
C67     3520
C64     3493
C34     1920
C32     1912
C02     1219
C19     1218
C00     1022
C25      902
C54      836
C62      820
C15      731
C53      615
C60      551
C22      486
C06      483
C49      464
C04      433
C17      420
C24      412
C56      387
C05      372
C07      287
C23      280
C09      208
C01      195
C40      188
C10      187
C51      182
C21      160
C48      132
C65      129
C03      125
C12      116
C69      108
C13      107
C66      100
C41       93
C08       89
C31       72
C30       61
C11       54
C38       22
C52       17
C47       15
C68       12
C63        4
C74        3
C57        2
Name: TOPOGRUP, dtype: int64

In [ ]:
df_aux.TOPOGRUP.value_counts().head(10)

C44    214251
C50    127597
C61    100165
C53     45784
C34     44507
C16     35745
C42     35574
C18     34933
C20     26979
C73     21141
Name: TOPOGRUP, dtype: int64

*   **MORFO**: Código da morfologia (char = 5). Formato: 99999

In [ ]:
df_grupo.MORFO.value_counts().head(15)

80903    33474
80703    23683
81403    12447
80973    10214
82113     6650
85003     5061
85503     4635
80923     4277
80702     2644
82603     2642
87203     2451
80913     2117
83103     1923
83123     1445
80103     1288
Name: MORFO, dtype: int64

In [ ]:
df_aux.MORFO.value_counts().head(15)

80703    152682
81403    131221
80903     97765
85003     92006
85503     64828
82113     28346
80973     23730
80103     23522
80702     17026
82603     10133
87203      9712
80003      8658
80102      8301
80503      7886
97323      7713
Name: MORFO, dtype: int64

*   **T**: Classificação TNM - T (char = 5). 
*   **N**: Classificação TNM - N (char = 5). 
*   **M**: Classificação TNM - M (char = 3). 


In [ ]:
df_grupo['T'].value_counts()

1        71964
2        19318
3        15956
IS        6302
1A        4433
4         3380
1B        2220
X         2184
2B        2078
2A        2057
2C        1668
1C        1641
4A        1377
3A        1078
4B         761
A          735
3B         585
0          129
CDIS       123
4D          41
CLIS        27
1MIC        25
4C          10
3C           8
PAGET        2
4E           1
Name: T, dtype: int64

In [ ]:
df_aux['T'].value_counts()

1        211140
X        152837
Y        120431
2        112052
3        107138
4         56554
1C        26653
IS        23265
4A        19815
2C        19425
4B        19087
1A        14889
2A        14082
2B        12591
1B         9619
3A         8567
3B         5818
A          2683
4D         2518
CDIS       1886
0          1408
4C          746
1MIC        173
CLIS        168
3C           76
PAGET        30
4E            8
Name: T, dtype: int64

In [ ]:
df_grupo.N.value_counts()

0      121163
1        9298
2        2686
X        2262
1A        843
1B        553
3         385
2A        352
2B        329
1C         87
2C         71
3A         49
3B         22
3C          1
PN0         1
CN0         1
Name: N, dtype: int64

In [ ]:
df_aux.N.value_counts()

0      489481
X      158458
Y      120431
1       88490
2       43839
3       20300
2B       5872
2A       4609
2C       3743
1A       3035
1B       2778
3A       1070
3B        646
3C        552
1C        336
PN0        11
CN0         8
Name: N, dtype: int64

In [ ]:
df_grupo.M.value_counts()

0     136318
1       1483
1A       131
X        118
1B        49
1C         4
Name: M, dtype: int64

In [ ]:
df_aux.M.value_counts()

0     669181
Y     120431
1      98235
X      39923
1B      9656
1A      5393
1C       840
Name: M, dtype: int64

*   **PT**: Estadiamento pós cirúrgico (char = 5). 
*   **PN**: Estadiamento pós cirúrgico (char = 5). 
*   **PM**: Estadiamento pós cirúrgico (char = 3).

In [ ]:
df_grupo.PT.value_counts()

1                 71741
2                 19322
3                 15650
IS                 6220
1A                 4783
4                  3461
1B                 2726
2B                 2265
2A                 2102
2C                 1948
1C                 1544
4A                 1281
3A                 1237
0                   955
A                   723
3B                  684
4B                  661
X                   143
3C                  120
CDIS                118
1B1                  88
1A1                  85
Sem informação       79
1MIC                 46
1B2                  32
4D                   30
CLIS                 22
1A2                  19
4C                   16
4E                    1
PAGET                 1
Name: PT, dtype: int64

In [ ]:
df_aux.PT.value_counts()

Sem informação    520304
X                 108003
1                  88413
2                  45487
Y                  42202
3                  37362
IS                 14893
4                  12999
1C                 11326
2C                 10775
1A                  8289
1B                  7106
2B                  5445
2A                  5267
4A                  4658
3A                  4179
4B                  4100
0                   3286
3B                  2922
< ...               1895
A                   1822
CDIS                 645
4D                   551
3C                   465
1A1                  327
1B1                  320
1MIC                 189
4C                   127
1B2                  124
CLIS                  94
1A2                   63
PAGET                  6
T2                     2
4E                     2
T3B                    2
T1C                    2
AD                     1
T3                     1
T1B                    1
PIS                    1


In [ ]:
df_grupo.PN.value_counts()

0                 117652
1                   9298
X                   5493
2                   2707
1A                   843
1B                   581
2A                   367
3                    357
2B                   337
Sem informação       182
1C                    89
2C                    87
3A                    80
3B                    23
< ...                  5
3C                     2
Name: PN, dtype: int64

In [ ]:
df_aux.PN.value_counts()

Sem informação    524869
0                 191548
X                 125357
Y                  42073
1                  32485
2                  12607
3                   3517
2A                  2076
1A                  1918
< ...               1910
2B                  1865
1B                  1565
3A                   847
2C                   669
1C                   163
3B                   115
3C                    65
N0                     3
1B1                    2
N1                     1
N2B                    1
1B2                    1
N3                     1
PN0                    1
Name: PN, dtype: int64

In [ ]:
df_grupo.PM.value_counts()

0                 125003
Sem informação      6691
X                   4665
1                   1581
1A                   119
1B                    28
< .                   10
1C                     6
Name: PM, dtype: int64

In [ ]:
df_aux.PM.value_counts()

Sem informação    546925
0                 214326
X                 119640
Y                  41928
1                  16777
< .                 1922
1A                  1081
1B                   948
1C                   106
M0                     6
Name: PM, dtype: int64

*   **G**: Classificação TNM – G (Grau) (char = 5). 

      Domínio (exceto C40, C41, C381, C382, C383, C47, C48 e C49):
      0; 1; 2; 3; 4; 8 – NÃO SE APLICA; 9 – X

      Domínio (somente C40, C41, C381, C382, C383, C47, C48 e C49):
      ALTO; BAIXO; 8 – NÃO SE APLICA; 9 – X

In [ ]:
df_grupo.G.value_counts()

8    137743
9       359
4         1
Name: G, dtype: int64

In [ ]:
df_aux.G.value_counts()

8    903221
9     12770
3     10600
2      8549
1      4720
4      3799
Name: G, dtype: int64

*   **LOCALTNM**: Classificação TNM - Localização (int = 1).
      
      1 – SUPERIOR
      
      2 – MEDIO
      
      3 – INFERIOR
      
      8 – NÃO SE APLICA
      
      9 – X	


In [ ]:
df_grupo.LOCALTNM.value_counts()

8    138088
9        15
Name: LOCALTNM, dtype: int64

In [ ]:
df_aux.LOCALTNM.value_counts()

8    939048
2      2006
9      1384
1       625
3       596
Name: LOCALTNM, dtype: int64

*   **IDMITOTIC**: Classificação TNM – Índice Mitótico (int = 1).
      
      1 – ALTA
      
      2 – BAIXA
      
      8 – NÃO SE APLICA
      
      9 – X	

In [ ]:
df_grupo.IDMITOTIC.value_counts()

8    137981
2        95
1        27
Name: IDMITOTIC, dtype: int64

In [ ]:
df_aux.IDMITOTIC.value_counts()

8    943154
2       267
1       151
9        87
Name: IDMITOTIC, dtype: int64

*   **PSA**: Classificação TNM - PSA (int = 1).
      
      1 – MENOR QUE 10
      
      2 – MAIOR OU IGUAL A 10 E MENOR QUE 20
      
      3 – MAIOR OU IGUAL A 20
      
      8 – NÃO SE APLICA
      
      9 – X	

In [ ]:
df_grupo.PSA.value_counts()

8    137549
9       549
2         4
1         1
Name: PSA, dtype: int64

In [ ]:
df_aux.PSA.value_counts()

8    911113
1     17698
2      6282
3      5362
9      3204
Name: PSA, dtype: int64

*   **GLEASON**: Classificação TNM - Gleason (int = 1).
      
      1 – MENOR OU IGUAL A 6
      
      2 – IGUAL A 7
      
      3 – MAIOR OU IGUAL A 8
      
      8 – NÃO SE APLICA
      
      9 – X

In [ ]:
df_grupo.GLEASON.value_counts()

8    137549
2       354
3        98
9        60
1        42
Name: GLEASON, dtype: int64

In [ ]:
df_aux.GLEASON.value_counts()

8    911113
2     13794
1     10079
3      6282
9      2391
Name: GLEASON, dtype: int64

*   **NAOTRAT**: Código da razão para não realização do tratamento (int = 1).
      
      1 – RECUSA DO TRATAMENTO
      
      2 – DOENÇA AVANÇADA, FALTA DE CONDIÇÕES CLINICAS
      
      3 – OUTRAS DOENÇAS ASSOCIADAS
      
      4 – ABANDONO DE TRATAMENTO
      
      5 – OBITO POR CANCER
      
      6 – OBITO POR OUTRAS CAUSAS, SOE
      
      7 – OUTRAS
      
      8 – NÃO SE APLICA (CASO TENHA TRATAMENTO)
      
      9 – SEM INFORMAÇÃO

In [ ]:
df_grupo.NAOTRAT.value_counts()

8    137848
7       201
9        52
3         1
2         1
Name: NAOTRAT, dtype: int64

In [ ]:
df_aux.NAOTRAT.value_counts()

8    868396
5     30655
7     20453
2      9166
9      7324
6      2474
3      1892
4      1718
1      1581
Name: NAOTRAT, dtype: int64

*   **TRATAMENTO**: Código de combinação dos tratamentos realizados (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento
      
      J – Nenhum tratamento realizado	


In [ ]:
df_grupo.TRATAMENTO.value_counts()

A    115270
E     10103
D      6310
C      2163
G      2144
B      1297
I       402
J       214
F       200
Name: TRATAMENTO, dtype: int64

In [ ]:
df_aux.TRATAMENTO.value_counts()

A    348265
I    152381
C     91449
J     74664
F     61147
E     59374
G     47249
B     46128
D     36372
H     26630
Name: TRATAMENTO, dtype: int64

*   **TRATHOSP**: Código de combinação dos tratamentos realizados no hospital (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento

      J – Nenhum tratamento realizado


In [ ]:
df_grupo.TRATHOSP.value_counts()

A    115359
E     10103
D      6310
C      2163
G      2144
B      1297
I       313
J       214
F       200
Name: TRATHOSP, dtype: int64

In [ ]:
df_aux.TRATHOSP.value_counts()

A    355515
I    151134
C     97661
J     78680
E     61222
F     56380
B     44720
G     42325
D     33016
H     23006
Name: TRATHOSP, dtype: int64

*   **TRATFAPOS**: Código de combinação dos tratamentos realizados após admissão fora do hospital (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento
      
      J – Nenhum tratamento realizado
      
      K – Sem informação


In [ ]:
df_grupo.TRATFAPOS.value_counts()

J    138014
I        89
Name: TRATFAPOS, dtype: int64

In [ ]:
df_aux.TRATFAPOS.value_counts()

J    900529
B     24186
I      8708
C      4887
A      2773
F      1611
D       379
E       353
G       174
H        59
Name: TRATFAPOS, dtype: int64

*   **NENHUM**: Tratamento recebido no hospital = nenhum (int = 1). 0 – NÃO; 1 – SIM
*   **CIRURGIA**: Tratamento recebido no hospital = cirurgia (int = 1). 0 – NÃO; 1 – SIM
*   **RADIO**: Tratamento recebido no hospital = radioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **QUIMIO**: Tratamento recebido no hospital = quimioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **HORMONIO**: Tratamento recebido no hospital = hormonioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **TMO**: Tratamento recebido no hospital = tmo (int = 1). 0 – NÃO; 1 – SIM
*   **IMUNO**: Tratamento recebido no hospital = imunoterapia (int = 1). 0 – NÃO; 1 – SIM
*   **OUTROS**: Tratamento recebido no hospital = outros (int = 1). 0 – NÃO; 1 – SIM

In [ ]:
df_grupo.NENHUM.value_counts()

0    137893
1       210
Name: NENHUM, dtype: int64

In [ ]:
df_aux.NENHUM.value_counts()

0    865018
1     78641
Name: NENHUM, dtype: int64

In [ ]:
df_grupo.CIRURGIA.value_counts()

1    134215
0      3888
Name: CIRURGIA, dtype: int64

In [ ]:
df_aux.CIRURGIA.value_counts()

1    588858
0    354801
Name: CIRURGIA, dtype: int64

In [ ]:
df_grupo.RADIO.value_counts()

0    128150
1      9953
Name: RADIO, dtype: int64

In [ ]:
df_aux.RADIO.value_counts()

0    698252
1    245407
Name: RADIO, dtype: int64

In [ ]:
df_grupo.QUIMIO.value_counts()

0    123491
1     14612
Name: QUIMIO, dtype: int64

In [ ]:
df_aux.QUIMIO.value_counts()

0    613077
1    330582
Name: QUIMIO, dtype: int64

In [ ]:
df_grupo.HORMONIO.value_counts()

0    138089
1        14
Name: HORMONIO, dtype: int64

In [ ]:
df_aux.HORMONIO.value_counts()

0    828493
1    115166
Name: HORMONIO, dtype: int64

In [ ]:
df_grupo.TMO.value_counts()

0    138099
1         4
Name: TMO, dtype: int64

In [ ]:
df_aux.TMO.value_counts()

0    939989
1      3670
Name: TMO, dtype: int64

In [ ]:
df_grupo.IMUNO.value_counts()

0    137808
1       295
Name: IMUNO, dtype: int64

In [ ]:
df_aux.IMUNO.value_counts()

0    937380
1      6279
Name: IMUNO, dtype: int64

In [ ]:
df_grupo.OUTROS.value_counts()

0    138103
Name: OUTROS, dtype: int64

In [ ]:
df_aux.OUTROS.value_counts()

0    883832
1     59827
Name: OUTROS, dtype: int64

*   **NENHUMAPOS**: Tratamento recebido fora do hospital e durante/após admissão = nenhum	(int = 1). 0 – NÃO; 1 – SIM		
*   **HORMOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = hormonioterapia	(int = 1). 0 – NÃO; 1 – SIM		
*   **TMOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = tmo	(int = 1). 0 – NÃO; 1 – SIM		
*   **IMUNOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = imunoterapia	(int = 1). 0 – NÃO; 1 – SIM	

In [ ]:
df_grupo.NENHUMAPOS.value_counts()

1    136586
0      1517
Name: NENHUMAPOS, dtype: int64

In [ ]:
df_aux.NENHUMAPOS.value_counts()

1    889063
0     54596
Name: NENHUMAPOS, dtype: int64

In [ ]:
df_grupo.HORMOAPOS.value_counts()

0    138027
1        76
Name: HORMOAPOS, dtype: int64

In [ ]:
df_aux.HORMOAPOS.value_counts()

0    941470
1      2189
Name: HORMOAPOS, dtype: int64

In [ ]:
df_grupo.TMOAPOS.value_counts()

0    138101
1         2
Name: TMOAPOS, dtype: int64

In [ ]:
df_aux.TMOAPOS.value_counts()

0    943421
1       238
Name: TMOAPOS, dtype: int64

In [ ]:
df_grupo.IMUNOAPOS.value_counts()

0    138092
1        11
Name: IMUNOAPOS, dtype: int64

In [ ]:
df_aux.IMUNOAPOS.value_counts()

0    943516
1       143
Name: IMUNOAPOS, dtype: int64

*   **ULTINFO**: Última informação sobre o paciente (int = 1).
      
      1 – VIVO, COM CÂNCER
      
      2 – VIVO, SOE
      
      3 – OBITO POR CANCER
      
      4 – OBITO POR OUTRAS CAUSAS, SOE

In [ ]:
df_grupo.ULTINFO.value_counts()

2    91788
4    25994
3    11864
1     8457
Name: ULTINFO, dtype: int64

In [ ]:
df_aux.ULTINFO.value_counts()

2    435227
3    275158
4    123266
1    110008
Name: ULTINFO, dtype: int64

*   **CICI**: Tumor infantil (char = 5).

In [ ]:
df_grupo.CICI.value_counts()

Sem informação    137873
XIE                   62
VIIIA                 35
XIF                   33
XC                    26
VIIIC                 20
VIIIB                  9
IXA                    8
XIB                    7
IXE                    6
IXD                    6
XID                    5
IXB                    4
VIIB                   3
VIB                    3
XIIA                   2
XIC                    1
Name: CICI, dtype: int64

In [ ]:
df_aux.CICI.value_counts()

Sem informação    918111
IA                  4468
IIA                 1919
IB                  1612
VIIIA               1239
IIB                 1237
IIIB                1216
IVA                 1108
VIA                 1081
VA                  1056
XIF                  885
IIIC                 792
IXA                  755
XC                   753
IIC                  655
VIIIC                631
XIB                  533
IXD                  532
IIID                 491
IIIA                 391
IIIE                 354
IXB                  322
XIE                  294
XB                   283
IE                   257
IC                   249
ID                   236
XIC                  214
VIIA                 204
XIA                  203
IXE                  195
XA                   191
XID                  137
IID                  112
IIE                  101
XIIB                  98
IIIF                  96
VIB                   89
XE                    88
VIIIB                 87


*   **CICIGRUP**: Tumor infantil – Grupo (char = 80).

In [ ]:
df_grupo.CICIGRUP.value_counts()

Sem informação    137873
XI                   108
VIII                  64
X                     26
IX                    24
VII                    3
VI                     3
XII                    2
Name: CICIGRUP, dtype: int64

In [ ]:
df_aux.CICIGRUP.value_counts()

Sem informação    918111
I                   6822
II                  4024
III                 3340
XI                  2266
VIII                2080
IX                  1806
X                   1380
VI                  1179
IV                  1159
V                   1056
VII                  294
XII                  142
Name: CICIGRUP, dtype: int64

*   **FAIXAETAR**: Faixa etária do paciente (char = 5).

      00-09 

      10-19

      20-29

      30-39

      40-49

      50-59

      60-69

      70+

In [ ]:
df_grupo.FAIXAETAR.value_counts()

70+      55954
60-69    36665
50-59    25982
40-49    12825
30-39     5016
20-29     1431
10-19      194
00-09       36
Name: FAIXAETAR, dtype: int64

In [ ]:
df_aux.FAIXAETAR.value_counts()

70+      287862
60-69    241092
50-59    196325
40-49    113183
30-39     54412
20-29     25002
00-09     13446
10-19     12337
Name: FAIXAETAR, dtype: int64

*   **RECNENHUM**: Sem recidiva (int = 1). 0 - Não; 1 - Sim	
*   **RECREGIO**: Recidiva regional (int = 1). 0 - Não; 1 - Sim	
*   **RECDIST**: Recidiva a distância / metástase (int = 1). 0 - Não; 1 - Sim

In [ ]:
df_grupo.RECNENHUM.value_counts()

1    137977
0       126
Name: RECNENHUM, dtype: int64

In [ ]:
df_aux.RECNENHUM.value_counts()

1    865619
0     78040
Name: RECNENHUM, dtype: int64

In [ ]:
df_grupo.RECLOCAL.value_counts()

0    138103
Name: RECLOCAL, dtype: int64

In [ ]:
df_aux.RECLOCAL.value_counts()

0    905605
1     38054
Name: RECLOCAL, dtype: int64

In [ ]:
df_grupo.RECREGIO.value_counts()

0    137988
1       115
Name: RECREGIO, dtype: int64

In [ ]:
df_aux.RECREGIO.value_counts()

0    920648
1     23011
Name: RECREGIO, dtype: int64

In [ ]:
df_grupo.RECDIST.value_counts()

0    138101
1         2
Name: RECDIST, dtype: int64

In [ ]:
df_aux.RECDIST.value_counts()

0    922659
1     21000
Name: RECDIST, dtype: int64